In [2]:
import requests
import json
from typing import List
base_url = "http://172.30.1.139:3000/api"

## Auth

In [3]:
user_api_key = "18f94155-086c-48b5-a210-59b545b4a039"  # HCIS API Key
headers = {
    "Authorization": f"users API-Key {user_api_key}",
}

## Create Tags If Not Exists

In [4]:


tags_to_be_used_in_created_scenario = [
    "behavior:cut-in",
    "party:hcis",
    "deliver:2024july",
    "src:cetran",
    "field:hct"
]

def create_tags_if_not_exists(tags: List[str]):
    tags_doc = requests.get(
        f"{base_url}/tags?depth=1&limit=1000000",
        headers=headers).json()["docs"]
    all_tags = [tag["id"] for tag in tags_doc]
    print(f"Existing tags: {all_tags}")

    for tag in tags:
        if tag not in all_tags:
            try:
                r = requests.post(
                    f"{base_url}/tags",
                    headers=headers,
                    json={
                        "id": tag,
                        "name": tag,
                        "description": tag  # optional
                    }
                )
                print(r.json())
            except Exception as e:
                print(e)
                print(f"Failed to create tag {tag}")


## Parameters
The parameters and ranges to be searched.

In [5]:
parameters = [  # Should be the same name using in the OpenSCENARIO file
    {
        "name": "TargetSpeed",
        "unit": "m/s",
        "min": 20,
        "max": 40,
    },
    {
        "name": "TargetLateralOffset",
        "unit": "m",
        "min": 1,
        "max": -1,
    },
    {

        "name": "TargetS",
        "unit": "m",
        "min": -5,
        "max": 5,
    }
]

## OpenSCENARIO File
2 methods to upload the openSCENARIO file

#### 1. Upload with xml strings
The field should be
```json
openScenarioField = {
    "type": "String",
    "content": "<openSCENARIO> <!-- Paste open scenario xml string here. --!> <openSCENARIO/>"
}
```

#### 2. Upload with a file
The field should be
```json
openScenarioField = {
    "type": "File",
    "openScenario": "OpenScenarioID Here"
}
```
To get the openScenarioID, you'll have to first upload the file to the endpoint `base_url/api/openScenarios`, get the id from the response, then use it in the openScenarioField

In [6]:
def use_scenario_string(content):
    return {
        "type": "String",
        "content": content,
    }

In [7]:
## Upload openScenario
def upload_openscenario(file_path):
    with open(file_path, "rb") as f:
        files = {
            "file": ("hct_04.xosc", f, "application/octet-stream")
        }
        r = requests.post(
            f"{base_url}/openScenarios",
            headers=headers,
            files=files,
        )
    scenario_id = r.json()["doc"]["id"]
    print(json.dumps(r.json(), indent=4))
    return {
        "type": "File",
        "openScenario": scenario_id
    }

## Routes
A route should be assigned to the scenario. This will be used to inform the autonomous system to switch route.

To add a route, please use the GUI [here](http://172.30.1.139:3000/admin/collections/routes?limit=10) or use the default route.

Then assign the ID of the route.

### Through API
Alternative way is to add a route and get the ID through API `http://172.39.1.139:3000/api/routes`.


In [21]:
route = "route-666c15f9173ee59246206343 hct-default"

## Tag Tree
Tag Tree follows the CETRAN format, with camel-style strings. Please refer to the example below.

In [9]:
tag_tree = {
    "ego": {
        "vehicleLongitudinalActivity": {
            "mode": "drivingForward",
            "drivingForwardMode": "cruising"
        },
        "vehicleLateralActivity": {
            "mode": "goingStraight"
        }
    },
    "actors": [
    {

        "vehicleLongitudinalActivity": {

            "mode": "drivingForward",
            "drivingForwardMode": "cruising"

        },
        "vehicleLateralActivity": {

            "mode": "goingStraight"

        },
        "initialState": {

            "direction": "oncoming",
            "dynamics": "standingStill",
            "lateralPosition": "leftOfEgo",
            "longitudinalPosition": "inFrontOfEgo"

        },
        "leadVehicle": {

            "mode": "appearing",
            "appearingMode": "gapClosing"

        },
    }
    ],
    "roadLayout": {
        "mode": "junction",
        "junctionMode": "noTrafficLight"
    }
}

## Test Objective
Is the key to evaluate whether a scenario is good enough.
Find all evaluation target at [keyPerformanceIndicators](http://172.30.1.139:3000/admin/collections/keyPerformanceIndicators?limit=1000).
Or please add new evaluation targets and tell how should I implement it.

In [10]:
test_objective = {
    "criticalityMetrics": [
        {
            "keyPerformanceIndicator": "666c0d22173ee5924620629a",  # "Collision Less Than"
            "threshold": 1,
            "description": "No collision allowed",
        },
        {
            "keyPerformanceIndicator": "666c0d33173ee592462062a2",  # "Max Deceleration Less Than"
            "threshold": 10,
            "id": "666c16533638a200018f9d21"
        }
    ]
}

## Valid Conditions
Only if the event(s) is(are) triggered will the trial be considered valid.
It should be the condition name used in the provided OpenSCENARIO.

In [11]:
valid_conditions = [
    {
        "condition": "TargetStartWhenEgoCloseToTheJunction"
    }
]

## Start ObservationSampling Conditions
After theses events start, the scenario will be recorded.

Uses same format as the valid conditions

In [12]:
start_conditions = [
    {
        "condition": "TargetStartWhenEgoCloseToTheJunction"
    },
    {
        "condition": "SomeOtherCondition"
    }
]

## Observation Recording Agents
The agents that should be recorded.

Left empty to record all agents.

In [13]:
observation_recording_agents = [
    {
        "name": "Target1"
    },
    {
        "name": "Target2"
    }
]

## Construct Request Body

In [15]:
def create_request_body(
            scenarioId: str,
            tags: List[str],
            description: str,
            parameters: List[dict],
            openDrive: str,
            openScenario: str,
            usedRoute: str,
            tagTree: dict,
            testObjectives: dict,
            validConditions: List[dict],
            startObservationSamplingConditions: List[dict],
            observationRecordingAgents: List[dict],
            egoTargetSpeed: float
        ):
    return {
        "id": scenarioId,
        "tags": tags,
        "description": description,
        "parameters": parameters,
        "openDrive": openDrive,
        "openScenarioField": openScenario,
        "usedRoute": usedRoute,
        "tagTree": tagTree,
        "testObjectives": testObjectives,
        "validConditions": validConditions,
        "startObservationSamplingConditions": startObservationSamplingConditions,
        "observationRecordingAgents": observationRecordingAgents,
        "egoTargetSpeed": egoTargetSpeed
    }

In [19]:
# openScenarioField = upload_openscenario("/resources/xosc/itri/hct_04.xosc")
openScenarioField = use_scenario_string("<OpenSCENARIO> <!-- some scenario --> </OpenSCENARIO>")
data = create_request_body(
    scenarioId = "test_scenario_from_api",
    tags = tags_to_be_used_in_created_scenario,
    description = "A vehicle crossing when the ego vehicle is about to enter the intersection.",
    parameters = parameters,
    openDrive = "666c15f9173ee59246206343",  # ID of hct_6.xodr
    openScenario = openScenarioField,
    usedRoute = route,
    tagTree = tag_tree,
    testObjectives = test_objective,
    validConditions = valid_conditions,
    startObservationSamplingConditions = start_conditions,
    observationRecordingAgents = observation_recording_agents,
    egoTargetSpeed = 40
)

In [20]:
r = requests.post(f"{base_url}/scenarios", headers=headers, json=data)
try:
    print(json.dumps(r.json(), indent=4))
except Exception as e:
    print(e)
except:
    print(r.text)

{
    "message": "Scenario successfully created.",
    "doc": {
        "id": "test_scenario_from_api",
        "createdBy": "HCIS Lab",
        "tags": [
            {
                "id": "behavior:cut-in",
                "name": "behavior:cut-in",
                "createdBy": "YC",
                "createdAt": "2024-06-17T07:57:46.897Z",
                "updatedAt": "2024-06-17T07:57:46.897Z"
            },
            {
                "id": "party:hcis",
                "name": "party:hcis",
                "createdBy": "showay",
                "createdAt": "2024-06-14T09:29:25.480Z",
                "updatedAt": "2024-06-19T02:51:54.895Z"
            },
            {
                "id": "deliver:2024july",
                "name": "deliver:2024july",
                "description": "deliver:2024july",
                "createdBy": "HCIS Lab",
                "createdAt": "2024-07-01T09:52:58.312Z",
                "updatedAt": "2024-07-01T09:52:58.312Z"
            },
         